In [1]:
import pandas as pd
from scipy import spatial
from openpyxl import load_workbook
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
import numpy as np

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [2]:
clean_filepath = "/Users/kylieleblancKylie/domino/docsim/data/clean/"
table_filepath = "/Users/kylieleblancKylie/domino/docsim/results/"

In [3]:
docs = pd.read_csv(clean_filepath + 'text_corpus.csv')
docs = docs.set_index('doc')
docs.sample(5)

,text,year,semester
doc,,,
7-2C.docx,What do you mean? What's not well? True. A...,2017-18,spring
2019_66_5C_Transcript.docx,"All right, so how do you think that went? ...",2018-19,spring
2018_97_3C_Transcript.docx,"All right, take a deep breath. All righ...",2018-19,fall
72-2C.docx,So how do you feel about that? Yeah? Eth...,2017-18,spring
53-2C.docx,"So, how do you feel about that first? Good? ...",2017-18,spring


In [5]:
matrix_corpus = pd.read_csv(clean_filepath + 'matrix_corpus.csv')
matrix_corpus = matrix_corpus.set_index('doc')
matrix_corpus.sample(5)

,10,100,12,15,16,20,29,30,40,7th,...,york,you,young,younger,your,yours,yourself,yup,zen,zone
doc,,,,,,,,,,,,,,,,,,,,,
2018_93_3C_Transcript.docx,0,0,0,0,0,0,0,0,0,0,...,0,40,0,0,6,0,0,0,0,0
2018_97_3C_Transcript.docx,0,0,1,0,0,0,0,0,0,0,...,1,27,0,0,0,0,0,0,0,0
2018_13_3C_Transcript.docx,0,0,0,0,0,0,0,0,0,0,...,0,13,0,0,3,0,0,0,0,0
2019_81_5C_Transcript.docx,0,0,0,0,0,0,0,0,0,0,...,0,39,0,0,5,0,0,0,0,0
2018_75_3C_Transcript.docx,0,0,0,0,0,0,0,0,0,0,...,0,39,0,0,3,0,0,0,0,0


In [6]:
def distance_to_mean(df, matrix_main, matrix_comparison):
    centroid = np.mean(matrix_comparison)
    sims = []
    for maindoc in matrix_main.index:
        sim = 1 - spatial.distance.cosine(matrix_main.loc[maindoc], centroid)
        sims.append(sim)
        #average = sum(pairwise_sim)/len(pairwise_sim)
        #df.at[maindoc, 'to_ideal'] = average
    
    return sims

In [26]:
matrix_main = matrix_corpus[matrix_corpus.index.isin(list(docs[(docs.year == '2018-19') & (docs.semester == 'spring')].index))]
matrix_comparison = matrix_corpus[matrix_corpus.index.isin(list(docs[(docs.year == '2017-18') & (docs.semester == 'spring')].index))]
cross_setting = distance_to_mean(docs, matrix_main, matrix_comparison)

In [27]:
matrix_main = matrix_corpus[matrix_corpus.index.isin(list(docs[(docs.year == '2018-19') & (docs.semester == 'spring')].index))]
matrix_comparison = matrix_corpus[matrix_corpus.index.isin(list(docs[(docs.year == '2018-19') & (docs.semester == 'fall')].index))]
cross_cohort = distance_to_mean(docs, matrix_main, matrix_comparison)

In [9]:
matrix_main = matrix_corpus[matrix_corpus.index.isin(list(docs[(docs.year == '2018-19') & (docs.semester == 'spring')].index))]
matrix_comparison = matrix_corpus[matrix_corpus.index.isin(list(docs[(docs.year == '2018-19') & (docs.semester == 'spring')].index))]
cross_nothing = distance_to_mean(docs, matrix_main, matrix_comparison)

AttributeError: 'list' object has no attribute 'mean'

In [29]:
cross_setting_mean = np.asarray(cross_setting).mean()
cross_setting_var = np.asarray(cross_setting).var()
cross_setting_weighted = (cross_setting_mean/cross_setting_var)
cross_setting_weight = 1/cross_setting_var
cross_setting_mean

0.8997673668974121

In [30]:
cross_cohort_mean = np.asarray(cross_cohort).mean()
cross_cohort_var = np.asarray(cross_cohort).var()
cross_cohort_weighted = (cross_cohort_mean/cross_cohort_var)
cross_cohort_weight = 1/cross_cohort_var
cross_cohort_mean

0.8581883958010609

In [10]:
cross_nothing_mean = np.asarray(cross_nothing).mean()
cross_nothing_var = np.asarray(cross_nothing).var()
cross_nothing_weighted = (cross_nothing_mean/cross_nothing_var)
cross_nothing_weight = 1/cross_nothing_var
cross_nothing_mean

0.9193659303115524

In [14]:
year = '2018-19'
semester = 'spring'
for maindoc in list(docs[(docs.year == year) & (docs.semester == semester)].index):
    pairwise_sim = []
    for doc in list(docs[(docs.year == year) & (docs.semester == semester)].index):
        sim = 1 - spatial.distance.cosine(matrix_corpus.loc[maindoc], matrix_corpus.loc[doc])
        pairwise_sim.append(sim)
    average = (sum(pairwise_sim) - 1)/(len(pairwise_sim) - 1) # don't include relationship with self
    average = (sum(pairwise_sim))/(len(pairwise_sim)) # don't include relationship with self
average

0.8191425502578794

In [32]:
tdot = (cross_setting_weighted + cross_cohort_weighted + cross_nothing_weighted)/(cross_setting_weight + cross_cohort_weight + cross_nothing_weight)
tdot

0.8938475415299135

In [33]:
cross_setting_q = (cross_setting_mean - tdot)**2/cross_setting_var
cross_cohort_q = (cross_cohort_mean - tdot)**2/cross_cohort_var
cross_nothing_q = (cross_nothing_mean - tdot)**2/cross_nothing_var

In [34]:
q = cross_setting_q + cross_cohort_q + cross_nothing_q
q

3.7627083924095803

In [35]:
# physics rule
# less than 1.25 = negligible heterogeneity
q/(3-1)

1.8813541962047902

In [36]:
# k = number of studies
# personnel psychology
# l <= (k-1)/3
# medicine
# l <= 2(k-1)/2

q = k - 1 + l <br/>
q - l = k - 1 <br/>
-l = k - 1 - q <br/>
l = -1(k - 1- q) <br/>
l = -k + 1 + q <br/>
l = q + 1 - k <br/>

In [37]:
l = q + 1 - 3
l

1.7627083924095803